# Kinematic generator
This notebook converts a Denavit Hartenberg table to its kinematic symbolic calculation and its jacobian.

In [1]:
from cmath import pi
import numpy as np
import sympy
# from sympy.abc import x,y,z,a,d
from kinematicBuilder import *

# import sys
# sys.set_int_max_str_digits(0)

# Examples

## Human arm model

### Human Arm (8 DOF)
Human arm modeled as 3 rotational joints for the shoulder (abduction, flexion, rotation), 2 joints for the elbow (flexion, pronation), 2 joints for the wrist and one of the hand

In [2]:
ar = 10
fa = 5
palm = 1
fing = 1.2

T_shz = DenavitRow(0    , 0  , 0    , -pi/2, Joint(sympy.Symbol('sh_z'),JointType.ROTATIONAL,upper_limit=165*pi/180, lower_limit=-pi/2))
T_shy = DenavitRow(pi/2 , 0  , 0    , pi/2,  Joint(sympy.Symbol('sh_y'),JointType.ROTATIONAL,upper_limit=pi/2,       lower_limit=-pi/2))
T_shx = DenavitRow(-pi/2, -ar, 0    , pi/2,  Joint(sympy.Symbol('sh_x'),JointType.ROTATIONAL,upper_limit=pi/2,       lower_limit=-pi/2))
T_elz = DenavitRow(0    , 0  , 0    , -pi/2, Joint(sympy.Symbol('el_z'),JointType.ROTATIONAL,upper_limit=165*pi/180, lower_limit=0))
T_elx = DenavitRow(0    , -fa, 0    , pi/2,  Joint(sympy.Symbol('el_x'),JointType.ROTATIONAL,upper_limit=pi/6,       lower_limit=-110*pi/180))
T_wrz = DenavitRow(pi/2 , 0  , 0    , -pi/2, Joint(sympy.Symbol('wr_z'),JointType.ROTATIONAL,upper_limit=10*pi/180,  lower_limit=-pi/6))
T_wry = DenavitRow(0    , 0  , -palm, 0,     Joint(sympy.Symbol('wr_y'),JointType.ROTATIONAL,upper_limit=pi/3,       lower_limit=-pi/2))
T_hdy = DenavitRow(0    , 0  , -fing, 0,     Joint(sympy.Symbol('hd_y'),JointType.ROTATIONAL,upper_limit=5*pi/180,   lower_limit=-pi/2))

In [3]:
T_arm8 = DenavitDK((T_shz,T_shy,T_shx,T_elz,T_elx,T_wrz,T_wry,T_hdy),"humanArm8")
T_arm8.genURDF()

### Human Arm (5 DOF)
Human arm modeled as 3 rotational joints for the shoulder (abduction, flexion, rotation), 2 joints for the elbow (flexion, pronation) and a fixed thumb

In [4]:
thumb = 0.8
T_thb = DenavitRow(0, 0, thumb, 0)

T_arm5 = DenavitDK((T_shz,T_shy,T_shx,T_elz,T_elx,T_thb),"humanArm5")
T_arm5.genURDF()

## 3 DOF Cartesian 
A 3 DOF Cartesian robot with 3 prismatic joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [5]:
T1 = DenavitRow(pi/2,   0, 0, pi/2,  Joint(sympy.Symbol('q_1'),JointType.PRISMATIC))
T2 = DenavitRow(pi/2,   0, 0, -pi/2, Joint(sympy.Symbol('q_2'),JointType.PRISMATIC))
T3 = DenavitRow(0,      0, 0, 0,     Joint(sympy.Symbol('q_3'),JointType.PRISMATIC))

T_cartesian = DenavitDK((T1,T2,T3),"Cartesian")

In [6]:
T_cartesian.inversePositionEval([0,0,0],[1,2,3])

0.0
Solution with error 0.0 in 1 iterations


array([ 3.,  1., -2.])

In [7]:
# T_cartesian.genURDF()

In [8]:
cartesian = T_cartesian.jacobian * T_cartesian.jointsSym
cartesian

Matrix([
[ q_2],
[-q_3],
[ q_1],
[   0],
[   0],
[   0]])

In [9]:
joint_values = (0, 0, 0)
T_cartesian.eval(joint_values)

array([[ 0, -1,  0,  0],
       [ 0,  0, -1,  0],
       [ 1,  0,  0,  0],
       [ 0,  0,  0,  1]])

## 3 DOF Articulated Robot
A 3 DOF Articulated robot with 3 rotational joints from [The Ultimate Guide to Jacobian Matrices for Robotics](https://automaticaddison.com/the-ultimate-guide-to-jacobian-matrices-for-robotics/)

In [10]:
T1 = DenavitRow(0,  1.5, 0,   pi/2, Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(0,  0,  1,  0,    Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(0,  0,  0.5,  0,    Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))

T_articular3 = DenavitDK((T1,T2,T3))

In [11]:
# A=T_articular3.directTransformSym
# U,S,V = A.singular_value_decomposition() # Memory error

In [12]:
joint_values = (0, 0, 0)
T_articular3.eval(joint_values)

array([[ 1. , -0. ,  0. ,  1.5],
       [ 0. , -0. , -1. ,  0. ],
       [ 0. ,  1. ,  0. ,  1.5],
       [ 0. ,  0. ,  0. ,  1. ]])

## 6 DOF articular
A 6 DOF articular robot that follows the following scheme: 
<!-- ![axes](axes_diagram.png) -->
<img src="axes_diagram.png" alt="axes_diagram" width="400"/>

In [13]:
L = [0.2, 1, 1, 0.2]
T1 = DenavitRow(0,      L[0],   0,      pi/2,   Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL,upper_limit=pi/2,lower_limit=-pi/2))
T2 = DenavitRow(pi/2,   0,      L[1],   0,      Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(pi,     0,      0,      pi/2,   Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))
# T_ = DenavitRow(pi/2,   0.02,       0,      pi/2)
T4 = DenavitRow(pi,     L[2],   0,      pi/2,   Joint(sympy.Symbol('q_4'),JointType.ROTATIONAL))
T5 = DenavitRow(pi,     0,      0,      pi/2,   Joint(sympy.Symbol('q_5'),JointType.ROTATIONAL))
T6 = DenavitRow(0,      L[3],   0,      0,      Joint(sympy.Symbol('q_6'),JointType.ROTATIONAL))

T_articular6 = DenavitDK((T1,T2,T3,T4,T5,T6))
T_articular6.directTransformSym

Matrix([
[(((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_1) + cos(q_1)*cos(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656))*cos(q_4 + 884279719003555/281474976710656) + sin(q_1)*sin(q_4 + 884279719003555/281474976710656))*cos(q_5 + 884279719003555/281474976710656) + (sin(q_2 + 884279719003555/562949953421312)*cos(q_1)*cos(q_3 + 884279719003555/281474976710656) + sin(q_3 + 884279719003555/281474976710656)*cos(q_1)*cos(q_2 + 884279719003555/562949953421312))*sin(q_5 + 884279719003555/281474976710656))*cos(q_6) + ((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_1) + cos(q_1)*cos(q_2 + 884279719003555/562949953421312)*cos(q_3 + 884279719003555/281474976710656))*sin(q_4 + 884279719003555/281474976710656) - sin(q_1)*cos(q_4 + 884279719003555/281474976710656))*sin(q_6), -(((-sin(q_2 + 884279719003555/562949953421312)*sin(q_3 + 884279719003555/281474976710656)*cos(q_

In [14]:
# T_articular6.genURDF()
# T_articular6.genCCode()

In [15]:
pose = T_articular6.eval((1,1,1,1,1,1))
pose

array([[ 0.21667169, -0.26112258,  0.94067443, -0.04166873],
       [-0.95867353, -0.23890322,  0.1545002 , -0.32699789],
       [ 0.18438665, -0.93527549, -0.30209487, -0.2294141 ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [16]:
T_articular6.inverseEval([0,0,0,0,0,0],pose)

0.5303672509456662
0.13309089704522306
0.19805321952667992
0.05809623585716697
0.017616481038507496
0.0009623456241788129
1.787629733289395e-06
Solution with error 1.787629733289395e-06 in 7 iterations
18.121146966473756
2.144665595977677
0.2512664394157346
0.0038410540557825236
8.995987982353852e-07
Solution with error 8.995987982353852e-07 in 5 iterations


[4.141116140768799,
 5.282429591444287,
 2.1409939196888277,
 4.1415146764314565,
 0.9993505753336114,
 0.9989170096848352]

## 6 DOF robot with transformations in between

In [17]:
Lv = 50
T1 = DenavitRow(0,      0,          0,     -pi/2,   Joint(sympy.Symbol('q_1'),JointType.ROTATIONAL))
T2 = DenavitRow(0,      0,          Lv,     0,      Joint(sympy.Symbol('q_2'),JointType.ROTATIONAL))
T3 = DenavitRow(0,      -0.0425,    Lv,     pi/2,   Joint(sympy.Symbol('q_3'),JointType.ROTATIONAL))
T_ = DenavitRow(pi/2,   0.02,       0,      pi/2)
T4 = DenavitRow(pi/2,   0,          0,      pi/2,   Joint(sympy.Symbol('q_4'),JointType.ROTATIONAL))
T5 = DenavitRow(pi/2,   0,          0,      pi/2,   Joint(sympy.Symbol('q_5'),JointType.ROTATIONAL))
T6 = DenavitRow(0,      0,          0,      0,      Joint(sympy.Symbol('q_6'),JointType.ROTATIONAL))
TCP = DenavitRow(0,     0,          10,     0)

T_walter = DenavitDK((T1,T2,T3,T_,T4,T5,T6,TCP),"Walter")

In [18]:
T_walter.genURDF()
# T_walter.genCCode()

In [19]:
# A=T_walter.directTransformSym
# U,S,V = A.singular_value_decomposition()

In [20]:
pose = T_walter.eval((1,1,1,1,1,1))
joints = T_walter.inverseEval([0,0,0,0,0,0],pose)
joints

833.7461155046242
214.28174711164618
59.02379479132681
9.988013375618783
0.258650977688607
4.803857948268533e-05
Solution with error 4.803857948268533e-05 in 6 iterations
31501.757214589757
8698.474095096368
6547.043251137708
24126.787542586535
15316.6375162577
28168.49069049724
13308.87168781936
121.0291465474434
530.5850591692912
88.84164580036354
13.644308563211178
45.53630372198886
229.12007607764096
693.9994757039447
51.83113027817151
2068.815339089401
2805.8213219679333
576.2993425924069
29.70832250028074
377.25613575221416
463.65001563100014
410.0970440129237
186.80734745042597
124.62741038257941
12.264742250141406
0.9821619404475482
0.010295812821268442
3.285188435835655e-07
Solution with error 3.285188435835655e-07 in 28 iterations


[4.127886933503795,
 2.1415903629941035,
 5.283975563317476,
 0.9932963611964022,
 2.145970535460075,
 4.134760618003171]

In [21]:
T_walter.eval(joints)

array([[-5.42919436e-01,  7.69282144e-01,  3.36813701e-01,
        -2.02960813e+00],
       [ 7.03985744e-01,  6.35588473e-01, -3.16909080e-01,
         1.22551404e+01],
       [-4.57867402e-01,  6.50559449e-02, -8.86636998e-01,
        -9.21252292e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [22]:
pose

array([[-5.42948980e-01,  7.69269352e-01,  3.36795292e-01,
        -2.02982706e+00],
       [ 7.03951467e-01,  6.35609326e-01, -3.16943396e-01,
         1.22555160e+01],
       [-4.57885069e-01,  6.50034461e-02, -8.86631725e-01,
        -9.21255942e+01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [23]:
position_walter = TwoLinkAndBase(Lv,Lv)
position_walter.directKinHomoTSymbols()
position_walter.directLambda(1,1,1)

array([[-0.2248451 , -0.84147098, -0.4912955 ,  3.35407432],
       [-0.35017549,  0.54030231, -0.7651474 ,  5.22366125],
       [ 0.90929743,  0.        , -0.41614684, 87.53842058],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [24]:
aux = DenavitDK((T1,T2,T3),"Walter")
aux.directLambdaTransform(1,1,1)

array([[ -0.2248451 ,  -0.84147098,   0.4912955 ,   3.38983683],
       [ -0.35017549,   0.54030231,   0.7651474 ,   5.2006984 ],
       [ -0.90929743,   0.        ,  -0.41614684, -87.53842058],
       [  0.        ,   0.        ,   0.        ,   1.        ]])